# Question 1: Card Game - Highest Card Wins

This notebook implements a simple card game where two players draw 5 cards each and compare their highest cards.

In [ ]:
import random

# Define the suits and ranks
#SUITS = ['♣', '♦', '♥', '♠'].     
SUITS = ['♣️', '♦️', '🖤', '♠️']

RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

# Create rank values for comparison (Ace is highest)
RANK_VALUES = {rank: idx for idx, rank in enumerate(RANKS)}

In [49]:
def generate_deck():
    """Generate a standard deck of 52 cards."""
    deck = []
    for suit in SUITS:
        for rank in RANKS:
            deck.append((rank, suit))
    return deck

# Generate the deck
deck = generate_deck()
print(f"Deck created with {len(deck)} cards")
print(f"First 5 cards: {deck[:5]}")

Deck created with 52 cards
First 5 cards: [('2', '♣️'), ('3', '♣️'), ('4', '♣️'), ('5', '♣️'), ('6', '♣️')]


In [50]:
def shuffle(deck):
    """Shuffle the deck in place."""
    random.shuffle(deck)
    return deck

def draw(deck, num_cards=1):
    """Draw cards from the deck.
    
    Args:
        deck: The deck to draw from
        num_cards: Number of cards to draw
    
    Returns:
        List of drawn cards
    """
    if len(deck) < num_cards:
        raise ValueError(f"Not enough cards in deck. Requested {num_cards}, but only {len(deck)} available.")
    
    drawn_cards = []
    for _ in range(num_cards):
        drawn_cards.append(deck.pop())
    return drawn_cards

In [51]:
class Player:
    """Represents a player with a hand of cards."""
    
    def __init__(self, name):
        self.name = name
        self.hand = []
    
    def draw_cards(self, deck, num_cards):
        """Draw cards from the deck into the player's hand."""
        cards = draw(deck, num_cards)
        self.hand.extend(cards)
    
    def get_highest_card(self):
        """Get the highest card in the player's hand."""
        if not self.hand:
            return None
        return max(self.hand, key=lambda card: RANK_VALUES[card[0]])
    
    def show_hand(self):
        """Display the player's hand."""
        cards_str = ', '.join([f"{rank}{suit}" for rank, suit in self.hand])
        return f"{self.name}'s hand: {cards_str}"

In [52]:
# Reset and shuffle the deck
deck = generate_deck()
shuffle(deck)
print(f"Deck shuffled! {len(deck)} cards ready.\n")

# Create two players
player1 = Player("Player 1")
player2 = Player("Player 2")

# Each player draws 5 cards
player1.draw_cards(deck, 5)
player2.draw_cards(deck, 5)

print(player1.show_hand())
print(player2.show_hand())
print(f"\nCards remaining in deck: {len(deck)}")

Deck shuffled! 52 cards ready.

Player 1's hand: 6🖤, K♣️, 5♠️, A🖤, 7♣️
Player 2's hand: J🖤, 5♣️, 9♦️, 2♦️, 3♣️

Cards remaining in deck: 42


In [53]:
# Compare highest cards
highest1 = player1.get_highest_card()
highest2 = player2.get_highest_card()

print(f"\n{player1.name}'s highest card: {highest1[0]}{highest1[1]} (value: {RANK_VALUES[highest1[0]]})")
print(f"{player2.name}'s highest card: {highest2[0]}{highest2[1]} (value: {RANK_VALUES[highest2[0]]})")

value1 = RANK_VALUES[highest1[0]]
value2 = RANK_VALUES[highest2[0]]

print("\n" + "="*50)
if value1 > value2:
    print(f"🎉 {player1.name} WINS!")
elif value2 > value1:
    print(f"🎉 {player2.name} WINS!")
else:
    print("🤝 IT'S A TIE!")
print("="*50)


Player 1's highest card: A🖤 (value: 12)
Player 2's highest card: J🖤 (value: 9)

🎉 Player 1 WINS!


# Question 2: Trick-Taking Card Game

## Part 2a: Game Rules

The game starts by picking one player to play first. These players will play each trick as follows:

1. The selected player plays a random card from their hand. This is the **starter**.

2. Every following player plays a card from their hand. This card **must be the same suit as the starter**. You can choose any valid card in this step. **Do not worry about player strategy**.

3. If a player does not have any cards in their hand of the same suit as the starter, **they may play any card**.

4. Print out each player and the card that they played, using the message "Player {X} played {Card}".

5. The winner is the player who played the card with the highest rank card that is also the **same suit as the starter**.

6. Print the name of the winner. The **winner of this trick is now the starting player** for the next trick.

7. This is repeated until all of the players have no cards in their hands, then the game ends.

**For this part of the problem, simulate an entire game.**

There should be 13 rounds total in the game.

In [ ]:
import random

# Card suits and ranks for Question 2
SUITS = ['♣️', '♦️', '🖤', '♠️']
CARD_RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

# Rank values for comparison
CARD_RANK_VALUES = {rank: idx for idx, rank in enumerate(CARD_RANKS)}

In [55]:
def create_deck():
    """Create a standard 52-card deck."""
    deck = []
    for suit in CARD_SUITS:
        for rank in CARD_RANKS:
            deck.append((rank, suit))
    return deck

def shuffle_deck(deck):
    """Shuffle the deck."""
    random.shuffle(deck)
    return deck

def deal_cards(deck, num_players, cards_per_player):
    """Deal cards to players.
    
    Args:
        deck: The deck to deal from
        num_players: Number of players
        cards_per_player: Cards each player receives
    
    Returns:
        List of hands (each hand is a list of cards)
    """
    hands = [[] for _ in range(num_players)]
    for i in range(cards_per_player):
        for player_idx in range(num_players):
            hands[player_idx].append(deck.pop())
    return hands

In [56]:
class TrickPlayer:
    """Represents a player in the trick-taking game."""
    
    def __init__(self, name, hand):
        self.name = name
        self.hand = hand
        self.tricks_won = 0
    
    def has_suit(self, suit):
        """Check if player has any cards of the given suit."""
        return any(card[1] == suit for card in self.hand)
    
    def play_card(self, starter_suit=None):
        """Play a card from hand.
        
        Args:
            starter_suit: The suit of the starter card (None if this player is starting)
        
        Returns:
            The card played (rank, suit)
        """
        if starter_suit is None:
            # Starting player - play random card
            card = random.choice(self.hand)
        elif self.has_suit(starter_suit):
            # Must play same suit if possible
            valid_cards = [card for card in self.hand if card[1] == starter_suit]
            card = random.choice(valid_cards)
        else:
            # Can play any card
            card = random.choice(self.hand)
        
        self.hand.remove(card)
        return card
    
    def has_cards(self):
        """Check if player has any cards left."""
        return len(self.hand) > 0

In [57]:
def play_trick(players, starting_player_idx):
    """Play one trick.
    
    Args:
        players: List of TrickPlayer objects
        starting_player_idx: Index of the player who starts this trick
    
    Returns:
        Index of the winning player
    """
    num_players = len(players)
    cards_played = []
    
    # Determine play order (starting with starting_player_idx)
    play_order = [(starting_player_idx + i) % num_players for i in range(num_players)]
    
    # First player plays the starter
    starter_idx = play_order[0]
    starter_card = players[starter_idx].play_card()
    starter_suit = starter_card[1]
    cards_played.append((starter_idx, starter_card))
    print(f"Player {starter_idx + 1} played {starter_card[0]}{starter_card[1]}")
    
    # Other players play in order
    for player_idx in play_order[1:]:
        card = players[player_idx].play_card(starter_suit)
        cards_played.append((player_idx, card))
        print(f"Player {player_idx + 1} played {card[0]}{card[1]}")
    
    # Determine winner (highest rank of starter suit)
    valid_plays = [(idx, card) for idx, card in cards_played if card[1] == starter_suit]
    winner_idx, winner_card = max(valid_plays, key=lambda x: CARD_RANK_VALUES[x[1][0]])
    
    print(f"\n→ Player {winner_idx + 1} wins the trick with {winner_card[0]}{winner_card[1]}!\n")
    players[winner_idx].tricks_won += 1
    
    return winner_idx

In [58]:
def simulate_game(num_players=4):
    """Simulate a complete trick-taking card game.
    
    Args:
        num_players: Number of players (default 4)
    """
    print("="*60)
    print(f"TRICK-TAKING CARD GAME - {num_players} Players")
    print("="*60)
    print()
    
    # Create and shuffle deck
    deck = create_deck()
    shuffle_deck(deck)
    
    # Deal cards (52 cards / 4 players = 13 cards each)
    cards_per_player = 52 // num_players
    hands = deal_cards(deck, num_players, cards_per_player)
    
    # Create players
    players = [TrickPlayer(f"Player {i+1}", hands[i]) for i in range(num_players)]
    
    # Show initial hands
    print("Initial hands:")
    for player in players:
        hand_str = ', '.join([f"{card[0]}{card[1]}" for card in player.hand])
        print(f"{player.name}: {hand_str}")
    print("\n" + "="*60)
    
    # Play all tricks - randomly select starting player
    current_starter = random.randint(0, num_players - 1)
    print(f"\nPlayer {current_starter + 1} will start the first trick!\n")
    trick_number = 1
    
    while players[0].has_cards():
        print(f"\nTRICK {trick_number}")
        print("-" * 60)
        current_starter = play_trick(players, current_starter)
        trick_number += 1
    
    # Show final results
    print("="*60)
    print("GAME OVER - Final Results")
    print("="*60)
    for player in players:
        print(f"{player.name}: {player.tricks_won} tricks won")
    
    # Determine overall winner
    winner = max(players, key=lambda p: p.tricks_won)
    print(f"\n🏆 {winner.name} wins the game with {winner.tricks_won} tricks!")
    print("="*60)

In [59]:
# Simulate a complete game with 4 players
simulate_game(num_players=4)

TRICK-TAKING CARD GAME - 4 Players

Initial hands:
Player 1: 7♠, 4♣, K♦, K♥, 3♦, 2♥, 6♦, 5♣, 10♣, J♠, 10♦, 8♥, 8♣
Player 2: 6♣, Q♠, 2♣, Q♣, 8♦, 7♣, 3♠, K♣, Q♥, 10♥, 4♥, J♥, 4♦
Player 3: 3♣, A♣, Q♦, 7♥, 5♦, 9♠, 2♦, J♣, 9♦, A♠, 10♠, A♦, J♦
Player 4: K♠, 6♥, 5♥, 6♠, 4♠, A♥, 2♠, 7♦, 8♠, 9♣, 9♥, 3♥, 5♠


Player 3 will start the first trick!


TRICK 1
------------------------------------------------------------
Player 3 played J♣
Player 4 played 9♣
Player 1 played 5♣
Player 2 played 6♣

→ Player 3 wins the trick with J♣!


TRICK 2
------------------------------------------------------------
Player 3 played A♠
Player 4 played 6♠
Player 1 played J♠
Player 2 played 3♠

→ Player 3 wins the trick with A♠!


TRICK 3
------------------------------------------------------------
Player 3 played A♦
Player 4 played 7♦
Player 1 played 10♦
Player 2 played 8♦

→ Player 3 wins the trick with A♦!


TRICK 4
------------------------------------------------------------
Player 3 played 7♥
Player 4 played A♥
Pla

## Part 2b: Scoring

Each round, the winning player gets a number of points based on the cards played on the table for that round.

- **5** is worth **5 fish points**
- **10** is worth **10 fish points**
- **K** is worth **10 fish points**
- **All other cards** are worth **0 points**

### Modify your code to support the following:

1. At the end of each round, print the number of points that the winner took.
2. At the end of the game, print each player's total number of points.
3. Print the name of any player with highest number of fish points.

In [60]:
# Point values for scoring
POINT_VALUES = {
    '5': 5,
    '10': 10,
    'K': 10
}

def calculate_trick_points(cards_played):
    """Calculate the total points from cards played in a trick.
    
    Args:
        cards_played: List of (player_idx, card) tuples
    
    Returns:
        Total points in the trick
    """
    total_points = 0
    for _, card in cards_played:
        rank = card[0]
        total_points += POINT_VALUES.get(rank, 0)
    return total_points

In [61]:
class ScoringPlayer:
    """Represents a player in the trick-taking game with scoring."""
    
    def __init__(self, name, hand):
        self.name = name
        self.hand = hand
        self.tricks_won = 0
        self.fish_points = 0
    
    def has_suit(self, suit):
        """Check if player has any cards of the given suit."""
        return any(card[1] == suit for card in self.hand)
    
    def play_card(self, starter_suit=None):
        """Play a card from hand.
        
        Args:
            starter_suit: The suit of the starter card (None if this player is starting)
        
        Returns:
            The card played (rank, suit)
        """
        if starter_suit is None:
            # Starting player - play random card
            card = random.choice(self.hand)
        elif self.has_suit(starter_suit):
            # Must play same suit if possible
            valid_cards = [card for card in self.hand if card[1] == starter_suit]
            card = random.choice(valid_cards)
        else:
            # Can play any card
            card = random.choice(self.hand)
        
        self.hand.remove(card)
        return card
    
    def add_points(self, points):
        """Add points to player's score."""
        self.fish_points += points
    
    def has_cards(self):
        """Check if player has any cards left."""
        return len(self.hand) > 0

In [62]:
def play_trick_with_scoring(players, starting_player_idx):
    """Play one trick with scoring.
    
    Args:
        players: List of ScoringPlayer objects
        starting_player_idx: Index of the player who starts this trick
    
    Returns:
        Index of the winning player
    """
    num_players = len(players)
    cards_played = []
    
    # Determine play order (starting with starting_player_idx)
    play_order = [(starting_player_idx + i) % num_players for i in range(num_players)]
    
    # First player plays the starter
    starter_idx = play_order[0]
    starter_card = players[starter_idx].play_card()
    starter_suit = starter_card[1]
    cards_played.append((starter_idx, starter_card))
    print(f"Player {starter_idx + 1} played {starter_card[0]}{starter_card[1]}")
    
    # Other players play in order
    for player_idx in play_order[1:]:
        card = players[player_idx].play_card(starter_suit)
        cards_played.append((player_idx, card))
        print(f"Player {player_idx + 1} played {card[0]}{card[1]}")
    
    # Determine winner (highest rank of starter suit)
    valid_plays = [(idx, card) for idx, card in cards_played if card[1] == starter_suit]
    winner_idx, winner_card = max(valid_plays, key=lambda x: CARD_RANK_VALUES[x[1][0]])
    
    # Calculate points from this trick
    trick_points = calculate_trick_points(cards_played)
    players[winner_idx].add_points(trick_points)
    players[winner_idx].tricks_won += 1
    
    print(f"\n→ Player {winner_idx + 1} wins the trick with {winner_card[0]}{winner_card[1]}!")
    print(f"→ Player {winner_idx + 1} earns {trick_points} fish points this round\n")
    
    return winner_idx

In [63]:
def simulate_game_with_scoring(num_players=4):
    """Simulate a complete trick-taking card game with scoring.
    
    Args:
        num_players: Number of players (default 4)
    """
    print("="*60)
    print(f"TRICK-TAKING CARD GAME WITH SCORING - {num_players} Players")
    print("="*60)
    print()
    
    # Create and shuffle deck
    deck = create_deck()
    shuffle_deck(deck)
    
    # Deal cards (52 cards / 4 players = 13 cards each)
    cards_per_player = 52 // num_players
    hands = deal_cards(deck, num_players, cards_per_player)
    
    # Create players with scoring
    players = [ScoringPlayer(f"Player {i+1}", hands[i]) for i in range(num_players)]
    
    # Show initial hands
    print("Initial hands:")
    for player in players:
        hand_str = ', '.join([f"{card[0]}{card[1]}" for card in player.hand])
        print(f"{player.name}: {hand_str}")
    print("\n" + "="*60)
    
    # Play all tricks - randomly select starting player
    current_starter = random.randint(0, num_players - 1)
    print(f"\nPlayer {current_starter + 1} will start the first trick!\n")
    trick_number = 1
    
    while players[0].has_cards():
        print(f"\nTRICK {trick_number}")
        print("-" * 60)
        current_starter = play_trick_with_scoring(players, current_starter)
        trick_number += 1
    
    # Show final results
    print("="*60)
    print("GAME OVER - Final Results")
    print("="*60)
    for player in players:
        print(f"{player.name}: {player.tricks_won} tricks won, {player.fish_points} fish points")
    
    # Determine overall winner by fish points
    max_points = max(player.fish_points for player in players)
    winners = [player for player in players if player.fish_points == max_points]
    
    print(f"\n🏆 Winner(s) with {max_points} fish points:")
    for winner in winners:
        print(f"   {winner.name}")
    print("="*60)

In [64]:
# Simulate a complete game with 4 players and scoring
simulate_game_with_scoring(num_players=4)

TRICK-TAKING CARD GAME WITH SCORING - 4 Players

Initial hands:
Player 1: 2♥, 3♣, 3♠, 4♠, J♦, 6♠, 7♠, 4♦, 6♦, Q♦, 5♣, 9♠, 5♥
Player 2: 8♠, Q♥, 10♦, 6♣, 4♣, K♣, Q♠, Q♣, 5♠, 10♠, K♠, A♠, J♣
Player 3: 2♣, 5♦, 7♦, K♥, 9♦, 7♣, 8♥, 10♥, 2♦, 9♣, J♥, J♠, 8♦
Player 4: 9♥, 3♥, 4♥, 2♠, A♣, 10♣, A♦, K♦, 6♥, A♥, 3♦, 7♥, 8♣


Player 2 will start the first trick!


TRICK 1
------------------------------------------------------------
Player 2 played 10♠
Player 3 played J♠
Player 4 played 2♠
Player 1 played 9♠

→ Player 3 wins the trick with J♠!
→ Player 3 earns 10 fish points this round


TRICK 2
------------------------------------------------------------
Player 3 played K♥
Player 4 played 3♥
Player 1 played 5♥
Player 2 played Q♥

→ Player 3 wins the trick with K♥!
→ Player 3 earns 15 fish points this round


TRICK 3
------------------------------------------------------------
Player 3 played 8♥
Player 4 played 6♥
Player 1 played 2♥
Player 2 played Q♣

→ Player 3 wins the trick with 8♥!
→ Player 3 ea

# Question 3: Poker Hand Validation

Given a set of six poker hand rules (flush, straight, full house, 4-of-a-kind, straight flush, royal flush), determine whether a given hand is valid by checking if it satisfies at least one of these rules.

## Poker Hand Definitions:

- **Flush**: All 5 cards have the same suit
- **Straight**: 5 cards in sequential rank order (e.g., 5-6-7-8-9)
- **Full House**: 3 cards of one rank and 2 cards of another rank (e.g., 3-3-3-K-K)
- **4-of-a-Kind**: 4 cards of the same rank (e.g., 9-9-9-9-3)
- **Straight Flush**: 5 cards in sequential rank order, all of the same suit
- **Royal Flush**: 10-J-Q-K-A all of the same suit

**Note**: Ace can be low (A-2-3-4-5) or high (10-J-Q-K-A) in straights.

In [65]:
from typing import List, Tuple, Set
from collections import Counter

# Poker card definitions
POKER_SUITS = ['♣', '♦', '♥', '♠']
POKER_RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
POKER_RANK_VALUES = {rank: idx for idx, rank in enumerate(POKER_RANKS)}

In [66]:
def is_flush(hand: List[Tuple[str, str]]) -> bool:
    """Check if all cards have the same suit."""
    suits = [card[1] for card in hand]
    return len(set(suits)) == 1

def is_straight(hand: List[Tuple[str, str]]) -> bool:
    """Check if cards are in sequential rank order."""
    ranks = [card[0] for card in hand]
    values = sorted([POKER_RANK_VALUES[rank] for rank in ranks])
    
    # Check normal straight
    is_sequential = all(values[i] + 1 == values[i + 1] for i in range(len(values) - 1))
    
    # Check for ace-low straight (A-2-3-4-5)
    ace_low_straight = sorted(values) == [0, 1, 2, 3, 12]  # 2,3,4,5,A
    
    return is_sequential or ace_low_straight

def is_full_house(hand: List[Tuple[str, str]]) -> bool:
    """Check if hand has 3 of one rank and 2 of another."""
    ranks = [card[0] for card in hand]
    rank_counts = Counter(ranks)
    counts = sorted(rank_counts.values())
    return counts == [2, 3]

def is_four_of_a_kind(hand: List[Tuple[str, str]]) -> bool:
    """Check if hand has 4 cards of the same rank."""
    ranks = [card[0] for card in hand]
    rank_counts = Counter(ranks)
    return 4 in rank_counts.values()

def is_straight_flush(hand: List[Tuple[str, str]]) -> bool:
    """Check if hand is both a straight and a flush."""
    return is_straight(hand) and is_flush(hand)

def is_royal_flush(hand: List[Tuple[str, str]]) -> bool:
    """Check if hand is 10-J-Q-K-A all of the same suit."""
    if not is_flush(hand):
        return False
    
    ranks = set(card[0] for card in hand)
    royal_ranks = {'10', 'J', 'Q', 'K', 'A'}
    return ranks == royal_ranks

In [67]:
def validate_poker_hand(hand: List[Tuple[str, str]]) -> Tuple[bool, List[str]]:
    """Determine if a poker hand is valid by checking all rules.
    
    Args:
        hand: List of 5 cards as (rank, suit) tuples
    
    Returns:
        Tuple of (is_valid, list of matching hand types)
    """
    if len(hand) != 5:
        return False, []
    
    matching_hands = []
    
    # Check in order from highest to lowest
    if is_royal_flush(hand):
        matching_hands.append("Royal Flush")
    if is_straight_flush(hand):
        matching_hands.append("Straight Flush")
    if is_four_of_a_kind(hand):
        matching_hands.append("Four of a Kind")
    if is_full_house(hand):
        matching_hands.append("Full House")
    if is_flush(hand):
        matching_hands.append("Flush")
    if is_straight(hand):
        matching_hands.append("Straight")
    
    is_valid = len(matching_hands) > 0
    return is_valid, matching_hands

def display_hand(hand: List[Tuple[str, str]]) -> str:
    """Display a hand in readable format."""
    return ', '.join([f"{rank}{suit}" for rank, suit in hand])

In [68]:
# Test cases
test_hands = [
    # Royal Flush
    ([('10', '♥'), ('J', '♥'), ('Q', '♥'), ('K', '♥'), ('A', '♥')], "Royal Flush"),
    
    # Straight Flush
    ([('5', '♠'), ('6', '♠'), ('7', '♠'), ('8', '♠'), ('9', '♠')], "Straight Flush"),
    
    # Four of a Kind
    ([('9', '♥'), ('9', '♦'), ('9', '♣'), ('9', '♠'), ('3', '♥')], "Four of a Kind"),
    
    # Full House
    ([('3', '♥'), ('3', '♦'), ('3', '♣'), ('K', '♠'), ('K', '♥')], "Full House"),
    
    # Flush
    ([('2', '♦'), ('5', '♦'), ('9', '♦'), ('J', '♦'), ('K', '♦')], "Flush"),
    
    # Straight
    ([('5', '♥'), ('6', '♦'), ('7', '♣'), ('8', '♠'), ('9', '♥')], "Straight"),
    
    # Ace-low Straight
    ([('A', '♥'), ('2', '♦'), ('3', '♣'), ('4', '♠'), ('5', '♥')], "Ace-low Straight"),
    
    # Invalid hand (no pattern)
    ([('2', '♥'), ('5', '♦'), ('9', '♣'), ('J', '♠'), ('K', '♥')], "Invalid (no pattern)"),
]

print("="*70)
print("POKER HAND VALIDATION TESTS")
print("="*70)

for hand, description in test_hands:
    is_valid, hand_types = validate_poker_hand(hand)
    hand_display = display_hand(hand)
    
    print(f"\nHand: {hand_display}")
    print(f"Description: {description}")
    print(f"Valid: {is_valid}")
    if hand_types:
        print(f"Hand Type(s): {', '.join(hand_types)}")
    print("-" * 70)

POKER HAND VALIDATION TESTS

Hand: 10♥, J♥, Q♥, K♥, A♥
Description: Royal Flush
Valid: True
Hand Type(s): Royal Flush, Straight Flush, Flush, Straight
----------------------------------------------------------------------

Hand: 5♠, 6♠, 7♠, 8♠, 9♠
Description: Straight Flush
Valid: True
Hand Type(s): Straight Flush, Flush, Straight
----------------------------------------------------------------------

Hand: 9♥, 9♦, 9♣, 9♠, 3♥
Description: Four of a Kind
Valid: True
Hand Type(s): Four of a Kind
----------------------------------------------------------------------

Hand: 3♥, 3♦, 3♣, K♠, K♥
Description: Full House
Valid: True
Hand Type(s): Full House
----------------------------------------------------------------------

Hand: 2♦, 5♦, 9♦, J♦, K♦
Description: Flush
Valid: True
Hand Type(s): Flush
----------------------------------------------------------------------

Hand: 5♥, 6♦, 7♣, 8♠, 9♥
Description: Straight
Valid: True
Hand Type(s): Straight
----------------------------------------

## Follow-up 1: Wildcards (Jokers)

Modify the approach to account for the presence of wildcards (Jokers) that can represent any card.

In [69]:
def generate_possible_hands(hand: List[Tuple[str, str]]) -> List[List[Tuple[str, str]]]:
    """Generate all possible hands by substituting Jokers with actual cards.
    
    Args:
        hand: List of cards including Jokers (represented as ('Joker', 'Joker'))
    
    Returns:
        List of all possible hands with Jokers substituted
    """
    joker_indices = [i for i, card in enumerate(hand) if card[0] == 'Joker']
    
    if not joker_indices:
        return [hand]
    
    # For simplicity, generate a subset of possible substitutions
    # In practice, you'd need to check all combinations
    possible_hands = []
    
    # Generate all possible cards
    all_cards = [(rank, suit) for suit in POKER_SUITS for rank in POKER_RANKS]
    
    # For one joker case (simplified)
    if len(joker_indices) == 1:
        joker_idx = joker_indices[0]
        for replacement_card in all_cards:
            new_hand = hand.copy()
            new_hand[joker_idx] = replacement_card
            possible_hands.append(new_hand)
    
    # For multiple jokers, would need nested loops or recursion
    # Simplified version: try common winning combinations
    
    return possible_hands if possible_hands else [hand]

def validate_poker_hand_with_jokers(hand: List[Tuple[str, str]]) -> Tuple[bool, List[str], List[Tuple[str, str]]]:
    """Validate a poker hand that may contain Jokers.
    
    Args:
        hand: List of 5 cards, may include Jokers
    
    Returns:
        Tuple of (is_valid, list of hand types, best hand configuration)
    """
    # If no jokers, use regular validation
    if not any(card[0] == 'Joker' for card in hand):
        is_valid, hand_types = validate_poker_hand(hand)
        return is_valid, hand_types, hand
    
    # Try to find the best hand by testing possible substitutions
    non_joker_cards = [card for card in hand if card[0] != 'Joker']
    num_jokers = len([card for card in hand if card[0] == 'Joker'])
    
    # Strategy: Try to complete the best possible hand
    # Check if we can make a straight, flush, etc.
    
    # Simple heuristic: Check suit distribution
    suits = [card[1] for card in non_joker_cards]
    suit_counts = Counter(suits)
    
    # Check rank distribution
    ranks = [card[0] for card in non_joker_cards]
    rank_counts = Counter(ranks)
    
    best_hand = hand
    best_hand_types = []
    
    # Try to make flush (if most cards are same suit)
    if suit_counts and max(suit_counts.values()) + num_jokers >= 5:
        most_common_suit = max(suit_counts, key=suit_counts.get)
        test_hand = non_joker_cards.copy()
        # Add jokers as cards of the most common suit
        for i in range(num_jokers):
            # Pick ranks that don't conflict
            for rank in POKER_RANKS:
                if (rank, most_common_suit) not in test_hand:
                    test_hand.append((rank, most_common_suit))
                    break
        
        is_valid, hand_types = validate_poker_hand(test_hand[:5])
        if is_valid and (not best_hand_types or len(hand_types) > len(best_hand_types)):
            best_hand = test_hand[:5]
            best_hand_types = hand_types
    
    # Try to make four of a kind (if we have pairs/trips)
    if rank_counts and max(rank_counts.values()) + num_jokers >= 4:
        most_common_rank = max(rank_counts, key=rank_counts.get)
        test_hand = non_joker_cards.copy()
        # Add jokers as cards of the most common rank
        for i in range(num_jokers):
            for suit in POKER_SUITS:
                if (most_common_rank, suit) not in test_hand:
                    test_hand.append((most_common_rank, suit))
                    break
        
        is_valid, hand_types = validate_poker_hand(test_hand[:5])
        if is_valid and "Four of a Kind" in hand_types:
            best_hand = test_hand[:5]
            best_hand_types = hand_types
    
    is_valid = len(best_hand_types) > 0
    return is_valid, best_hand_types, best_hand

In [70]:
# Test with Jokers
joker_hands = [
    # One joker completing a flush
    ([('Joker', 'Joker'), ('5', '♥'), ('7', '♥'), ('9', '♥'), ('J', '♥')], "Joker + 4 hearts"),
    
    # Two jokers completing four of a kind
    ([('Joker', 'Joker'), ('Joker', 'Joker'), ('K', '♥'), ('K', '♦'), ('3', '♠')], "2 Jokers + pair of Kings"),
    
    # One joker completing a straight
    ([('Joker', 'Joker'), ('6', '♦'), ('7', '♣'), ('8', '♠'), ('9', '♥')], "Joker + 6-7-8-9"),
]

print("="*70)
print("POKER HAND VALIDATION WITH JOKERS")
print("="*70)

for hand, description in joker_hands:
    is_valid, hand_types, best_hand = validate_poker_hand_with_jokers(hand)
    hand_display = display_hand(hand)
    best_hand_display = display_hand(best_hand)
    
    print(f"\nOriginal Hand: {hand_display}")
    print(f"Description: {description}")
    print(f"Valid: {is_valid}")
    if hand_types:
        print(f"Hand Type(s): {', '.join(hand_types)}")
        print(f"Best Configuration: {best_hand_display}")
    print("-" * 70)

POKER HAND VALIDATION WITH JOKERS

Original Hand: JokerJoker, 5♥, 7♥, 9♥, J♥
Description: Joker + 4 hearts
Valid: True
Hand Type(s): Flush
Best Configuration: 5♥, 7♥, 9♥, J♥, 2♥
----------------------------------------------------------------------

Original Hand: JokerJoker, JokerJoker, K♥, K♦, 3♠
Description: 2 Jokers + pair of Kings
Valid: True
Hand Type(s): Four of a Kind
Best Configuration: K♥, K♦, 3♠, K♣, K♠
----------------------------------------------------------------------

Original Hand: JokerJoker, 6♦, 7♣, 8♠, 9♥
Description: Joker + 6-7-8-9
Valid: False
----------------------------------------------------------------------


## Follow-up 2: Comparing Two Hands

Given two players' hands and an ordering of the poker hand rules, compare the two hands to determine which is better.

In [71]:
# Hand rankings from highest to lowest
HAND_RANKINGS = [
    "Royal Flush",
    "Straight Flush",
    "Four of a Kind",
    "Full House",
    "Flush",
    "Straight",
    "Three of a Kind",
    "Two Pair",
    "One Pair",
    "High Card"
]

def get_best_hand_type(hand_types: List[str]) -> str:
    """Get the best (highest ranking) hand type from a list."""
    for ranking in HAND_RANKINGS:
        if ranking in hand_types:
            return ranking
    return "High Card"

def get_high_card_value(hand: List[Tuple[str, str]]) -> int:
    """Get the highest card value in the hand."""
    ranks = [card[0] for card in hand]
    values = [POKER_RANK_VALUES[rank] for rank in ranks]
    return max(values)

def compare_hands(hand1: List[Tuple[str, str]], hand2: List[Tuple[str, str]]) -> Tuple[int, str]:
    """Compare two poker hands.
    
    Args:
        hand1: First player's hand
        hand2: Second player's hand
    
    Returns:
        Tuple of (winner, reason)
        winner: 1 if hand1 wins, 2 if hand2 wins, 0 if tie
        reason: Explanation of the result
    """
    is_valid1, hand_types1 = validate_poker_hand(hand1)
    is_valid2, hand_types2 = validate_poker_hand(hand2)
    
    # Get best hand type for each
    best_type1 = get_best_hand_type(hand_types1) if is_valid1 else "High Card"
    best_type2 = get_best_hand_type(hand_types2) if is_valid2 else "High Card"
    
    rank1 = HAND_RANKINGS.index(best_type1) if best_type1 in HAND_RANKINGS else len(HAND_RANKINGS)
    rank2 = HAND_RANKINGS.index(best_type2) if best_type2 in HAND_RANKINGS else len(HAND_RANKINGS)
    
    if rank1 < rank2:
        return 1, f"Player 1 wins with {best_type1} vs {best_type2}"
    elif rank2 < rank1:
        return 2, f"Player 2 wins with {best_type2} vs {best_type1}"
    else:
        # Same hand type, compare high cards
        high1 = get_high_card_value(hand1)
        high2 = get_high_card_value(hand2)
        
        if high1 > high2:
            return 1, f"Tie on {best_type1}, Player 1 wins with higher card ({POKER_RANKS[high1]} vs {POKER_RANKS[high2]})"
        elif high2 > high1:
            return 2, f"Tie on {best_type2}, Player 2 wins with higher card ({POKER_RANKS[high2]} vs {POKER_RANKS[high1]})"
        else:
            return 0, f"Complete tie: both have {best_type1} with same high card"

In [72]:
# Test hand comparisons
comparison_tests = [
    (
        [('10', '♥'), ('J', '♥'), ('Q', '♥'), ('K', '♥'), ('A', '♥')],  # Royal Flush
        [('5', '♠'), ('6', '♠'), ('7', '♠'), ('8', '♠'), ('9', '♠')]    # Straight Flush
    ),
    (
        [('9', '♥'), ('9', '♦'), ('9', '♣'), ('9', '♠'), ('3', '♥')],  # Four of a Kind
        [('3', '♥'), ('3', '♦'), ('3', '♣'), ('K', '♠'), ('K', '♥')]   # Full House
    ),
    (
        [('2', '♦'), ('5', '♦'), ('9', '♦'), ('J', '♦'), ('K', '♦')],  # Flush
        [('2', '♣'), ('5', '♠'), ('9', '♥'), ('J', '♦'), ('A', '♦')]   # High Card (Ace)
    ),
    (
        [('5', '♥'), ('6', '♦'), ('7', '♣'), ('8', '♠'), ('9', '♥')],  # Straight (9 high)
        [('6', '♥'), ('7', '♦'), ('8', '♣'), ('9', '♠'), ('10', '♥')]  # Straight (10 high)
    ),
]

print("="*70)
print("POKER HAND COMPARISONS")
print("="*70)

for i, (hand1, hand2) in enumerate(comparison_tests, 1):
    print(f"\n--- Comparison {i} ---")
    print(f"Player 1: {display_hand(hand1)}")
    is_valid1, types1 = validate_poker_hand(hand1)
    if types1:
        print(f"          {', '.join(types1)}")
    
    print(f"Player 2: {display_hand(hand2)}")
    is_valid2, types2 = validate_poker_hand(hand2)
    if types2:
        print(f"          {', '.join(types2)}")
    
    winner, reason = compare_hands(hand1, hand2)
    print(f"\nResult: {reason}")
    print("-" * 70)

POKER HAND COMPARISONS

--- Comparison 1 ---
Player 1: 10♥, J♥, Q♥, K♥, A♥
          Royal Flush, Straight Flush, Flush, Straight
Player 2: 5♠, 6♠, 7♠, 8♠, 9♠
          Straight Flush, Flush, Straight

Result: Player 1 wins with Royal Flush vs Straight Flush
----------------------------------------------------------------------

--- Comparison 2 ---
Player 1: 9♥, 9♦, 9♣, 9♠, 3♥
          Four of a Kind
Player 2: 3♥, 3♦, 3♣, K♠, K♥
          Full House

Result: Player 1 wins with Four of a Kind vs Full House
----------------------------------------------------------------------

--- Comparison 3 ---
Player 1: 2♦, 5♦, 9♦, J♦, K♦
          Flush
Player 2: 2♣, 5♠, 9♥, J♦, A♦

Result: Player 1 wins with Flush vs High Card
----------------------------------------------------------------------

--- Comparison 4 ---
Player 1: 5♥, 6♦, 7♣, 8♠, 9♥
          Straight
Player 2: 6♥, 7♦, 8♣, 9♠, 10♥
          Straight

Result: Tie on Straight, Player 2 wins with higher card (10 vs 9)
----------------

# Question 4: Team Card Game with Lives and Skips

A team of multiple players plays a card game with specific constraints:

## Game Rules:

- **Team Lives**: The team starts with `num_players + 1` lives
- **Skips Available**: The team can skip up to `x` rounds (input parameter)
- **Player Order**: Players play in a fixed order (determined at start)
- **Hand Management**: Each player's hand is always sorted in ascending order
- **Card Playing**: Players must play their smallest available card
- **Round Success**: A round succeeds if each player's card is strictly larger than the previous player's
- **Round Failure Options**:
  - Use a skip (if available) - Players draw new cards and no one plays that round
  - Lose a life
- **Total Rounds**: The game runs for Y rounds
- **Win Condition**: Complete all Y rounds without running out of lives

## Objective:

Determine whether the team can survive all Y rounds given:
- Number of players
- Number of skips available
- Number of rounds to play

In [73]:
import random
from typing import List, Tuple

class TeamPlayer:
    """Represents a player in the team card game."""
    
    def __init__(self, name: str, deck: List[int]):
        self.name = name
        self.deck = deck  # Personal deck to draw from
        self.hand = []
    
    def draw_cards(self, num_cards: int = 5):
        """Draw cards from personal deck and sort hand."""
        for _ in range(num_cards):
            if self.deck:
                self.hand.append(self.deck.pop(0))
        self.hand.sort()
    
    def play_smallest_card(self) -> int:
        """Play and remove the smallest card from hand."""
        if not self.hand:
            raise ValueError(f"{self.name} has no cards to play!")
        return self.hand.pop(0)
    
    def has_cards(self) -> bool:
        """Check if player has cards in hand or deck."""
        return len(self.hand) > 0 or len(self.deck) > 0
    
    def discard_hand(self):
        """Discard current hand."""
        self.hand = []

In [74]:
class TeamCardGame:
    """Manages the team card game with lives and skips."""
    
    def __init__(self, num_players: int, num_skips: int, num_rounds: int, 
                 cards_per_hand: int = 5, verbose: bool = True):
        """
        Initialize the team card game.
        
        Args:
            num_players: Number of players on the team
            num_skips: Number of skips available
            num_rounds: Total rounds to play
            cards_per_hand: Cards dealt per round
            verbose: Whether to print game progress
        """
        self.num_players = num_players
        self.lives = num_players + 1
        self.skips_remaining = num_skips
        self.num_rounds = num_rounds
        self.cards_per_hand = cards_per_hand
        self.verbose = verbose
        
        # Create players with individual decks
        self.players = []
        for i in range(num_players):
            # Each player gets their own shuffled deck of cards (1-100)
            deck = list(range(1, 101))
            random.shuffle(deck)
            self.players.append(TeamPlayer(f"Player {i+1}", deck))
    
    def play_round(self, round_num: int) -> bool:
        """
        Play one round of the game.
        
        Returns:
            True if round succeeded, False if round failed
        """
        if self.verbose:
            print(f"\n{'='*60}")
            print(f"ROUND {round_num}")
            print(f"Lives: {self.lives} | Skips: {self.skips_remaining}")
            print(f"{'='*60}")
        
        # Each player draws cards
        for player in self.players:
            player.draw_cards(self.cards_per_hand)
        
        if self.verbose:
            for player in self.players:
                print(f"{player.name}'s hand: {player.hand}")
        
        # Players play in order
        cards_played = []
        previous_card = -1  # Start with -1 so first card is always valid
        round_success = True
        
        if self.verbose:
            print(f"\nPlaying cards:")
        
        for player in self.players:
            card = player.play_smallest_card()
            cards_played.append((player.name, card))
            
            if card <= previous_card:
                round_success = False
                if self.verbose:
                    print(f"{player.name} played {card} ✗ (not greater than {previous_card})")
                break
            else:
                if self.verbose:
                    print(f"{player.name} played {card} ✓")
                previous_card = card
        
        return round_success
    
    def handle_failed_round(self, round_num: int) -> str:
        """
        Handle a failed round - use skip or lose life.
        
        Returns:
            Action taken: "skip" or "life"
        """
        # Strategy: Use skip if available, otherwise lose a life
        if self.skips_remaining > 0:
            self.skips_remaining -= 1
            if self.verbose:
                print(f"\n→ Round {round_num} FAILED! Using a skip.")
                print(f"→ Skips remaining: {self.skips_remaining}")
            # Discard hands and draw new cards
            for player in self.players:
                player.discard_hand()
            return "skip"
        else:
            self.lives -= 1
            if self.verbose:
                print(f"\n→ Round {round_num} FAILED! Losing a life.")
                print(f"→ Lives remaining: {self.lives}")
            return "life"
    
    def play_game(self) -> bool:
        """
        Play the complete game.
        
        Returns:
            True if team survives all rounds, False otherwise
        """
        if self.verbose:
            print(f"\n{'#'*60}")
            print(f"TEAM CARD GAME")
            print(f"Players: {self.num_players} | Lives: {self.lives} | Skips: {self.skips_remaining}")
            print(f"Total Rounds: {self.num_rounds}")
            print(f"{'#'*60}")
        
        for round_num in range(1, self.num_rounds + 1):
            # Check if team is still alive
            if self.lives <= 0:
                if self.verbose:
                    print(f"\n{'='*60}")
                    print(f"GAME OVER - Team ran out of lives at round {round_num}")
                    print(f"{'='*60}")
                return False
            
            # Play the round
            round_success = self.play_round(round_num)
            
            if round_success:
                if self.verbose:
                    print(f"\n→ Round {round_num} SUCCEEDED! ✓")
            else:
                self.handle_failed_round(round_num)
        
        # Check final result
        if self.lives > 0:
            if self.verbose:
                print(f"\n{'='*60}")
                print(f"VICTORY! Team completed all {self.num_rounds} rounds!")
                print(f"Final lives: {self.lives} | Final skips: {self.skips_remaining}")
                print(f"{'='*60}")
            return True
        else:
            if self.verbose:
                print(f"\n{'='*60}")
                print(f"DEFEAT! Team ran out of lives.")
                print(f"{'='*60}")
            return False

In [75]:
# Example 1: Small game with generous resources
print("Example 1: 3 players, 2 skips, 5 rounds")
game1 = TeamCardGame(num_players=3, num_skips=2, num_rounds=5)
result1 = game1.play_game()
print(f"\nResult: {'SUCCESS' if result1 else 'FAILURE'}")

Example 1: 3 players, 2 skips, 5 rounds

############################################################
TEAM CARD GAME
Players: 3 | Lives: 4 | Skips: 2
Total Rounds: 5
############################################################

ROUND 1
Lives: 4 | Skips: 2
Player 1's hand: [28, 30, 39, 49, 99]
Player 2's hand: [1, 37, 49, 50, 87]
Player 3's hand: [2, 29, 49, 69, 93]

Playing cards:
Player 1 played 28 ✓
Player 2 played 1 ✗ (not greater than 28)

→ Round 1 FAILED! Using a skip.
→ Skips remaining: 1

ROUND 2
Lives: 4 | Skips: 1
Player 1's hand: [13, 16, 48, 92, 93]
Player 2's hand: [4, 17, 32, 34, 48]
Player 3's hand: [48, 54, 67, 73, 98]

Playing cards:
Player 1 played 13 ✓
Player 2 played 4 ✗ (not greater than 13)

→ Round 2 FAILED! Using a skip.
→ Skips remaining: 0

ROUND 3
Lives: 4 | Skips: 0
Player 1's hand: [6, 25, 31, 51, 55]
Player 2's hand: [41, 42, 45, 60, 70]
Player 3's hand: [19, 21, 24, 27, 44]

Playing cards:
Player 1 played 6 ✓
Player 2 played 41 ✓
Player 3 played 19 ✗ (not

In [76]:
# Example 2: Harder game with limited resources
print("\n" + "="*70)
print("Example 2: 4 players, 1 skip, 8 rounds")
game2 = TeamCardGame(num_players=4, num_skips=1, num_rounds=8)
result2 = game2.play_game()
print(f"\nResult: {'SUCCESS' if result2 else 'FAILURE'}")


Example 2: 4 players, 1 skip, 8 rounds

############################################################
TEAM CARD GAME
Players: 4 | Lives: 5 | Skips: 1
Total Rounds: 8
############################################################

ROUND 1
Lives: 5 | Skips: 1
Player 1's hand: [3, 29, 68, 80, 94]
Player 2's hand: [2, 12, 38, 93, 100]
Player 3's hand: [15, 63, 92, 97, 100]
Player 4's hand: [33, 55, 65, 77, 81]

Playing cards:
Player 1 played 3 ✓
Player 2 played 2 ✗ (not greater than 3)

→ Round 1 FAILED! Using a skip.
→ Skips remaining: 0

ROUND 2
Lives: 5 | Skips: 0
Player 1's hand: [14, 22, 37, 51, 52]
Player 2's hand: [29, 30, 43, 67, 72]
Player 3's hand: [9, 48, 60, 64, 77]
Player 4's hand: [3, 71, 82, 87, 96]

Playing cards:
Player 1 played 14 ✓
Player 2 played 29 ✓
Player 3 played 9 ✗ (not greater than 29)

→ Round 2 FAILED! Losing a life.
→ Lives remaining: 4

ROUND 3
Lives: 4 | Skips: 0
Player 1's hand: [4, 22, 37, 51, 52, 63, 70, 76, 82]
Player 2's hand: [19, 25, 30, 32, 43, 61, 67,

In [77]:
# Example 3: Simulation to determine win probability
print("\n" + "="*70)
print("Example 3: Running 100 simulations (3 players, 2 skips, 10 rounds)")
print("="*70)

num_simulations = 100
wins = 0

for i in range(num_simulations):
    game = TeamCardGame(num_players=3, num_skips=2, num_rounds=10, verbose=False)
    if game.play_game():
        wins += 1

win_rate = (wins / num_simulations) * 100
print(f"\nSimulation Results:")
print(f"Wins: {wins}/{num_simulations}")
print(f"Win Rate: {win_rate:.1f}%")

if win_rate >= 50:
    print(f"\n✓ The team CAN likely survive with these parameters ({win_rate:.1f}% success rate)")
else:
    print(f"\n✗ The team will likely FAIL with these parameters ({win_rate:.1f}% success rate)")


Example 3: Running 100 simulations (3 players, 2 skips, 10 rounds)

Simulation Results:
Wins: 0/100
Win Rate: 0.0%

✗ The team will likely FAIL with these parameters (0.0% success rate)


## Analysis Function

Let's create a function to analyze whether a team can survive based on simulation.

In [78]:
def can_team_survive_theoretical(num_players: int, num_skips: int, num_rounds: int) -> dict:
    """
    Theoretical analysis of whether a team can survive.
    
    The team can afford to fail at most (lives + skips) times.
    Lives = num_players + 1
    Total allowed failures = lives + skips = (num_players + 1) + num_skips
    
    However, the actual probability depends on card distribution.
    This function provides theoretical bounds.
    
    Args:
        num_players: Number of players
        num_skips: Number of skips available
        num_rounds: Total rounds to play
    
    Returns:
        Dictionary with analysis results
    """
    lives = num_players + 1
    total_resources = lives + num_skips
    
    # Run a simulation to estimate failure rate
    num_test_simulations = 1000
    failures = 0
    
    for _ in range(num_test_simulations):
        game = TeamCardGame(num_players, num_skips, num_rounds, verbose=False)
        if not game.play_game():
            failures += 1
    
    failure_rate = failures / num_test_simulations
    success_rate = 1 - failure_rate
    
    return {
        'num_players': num_players,
        'lives': lives,
        'skips': num_skips,
        'total_resources': total_resources,
        'num_rounds': num_rounds,
        'max_allowed_failures': total_resources,
        'success_rate': success_rate,
        'can_survive': success_rate >= 0.5,
        'confidence': 'high' if abs(success_rate - 0.5) > 0.2 else 'medium' if abs(success_rate - 0.5) > 0.1 else 'low'
    }

In [79]:
# Test different scenarios
test_scenarios = [
    (3, 2, 5),   # Easy
    (3, 2, 10),  # Medium
    (4, 1, 8),   # Hard
    (5, 3, 15),  # Very Hard
    (2, 5, 10),  # Easy with many skips
]

print("="*80)
print("TEAM SURVIVAL ANALYSIS")
print("="*80)

for num_players, num_skips, num_rounds in test_scenarios:
    result = can_team_survive_theoretical(num_players, num_skips, num_rounds)
    
    print(f"\nScenario: {num_players} players, {num_skips} skips, {num_rounds} rounds")
    print(f"  Lives: {result['lives']}")
    print(f"  Total Resources (lives + skips): {result['total_resources']}")
    print(f"  Success Rate: {result['success_rate']*100:.1f}%")
    print(f"  Verdict: {'✓ CAN SURVIVE' if result['can_survive'] else '✗ WILL LIKELY FAIL'}")
    print(f"  Confidence: {result['confidence'].upper()}")

print("\n" + "="*80)

TEAM SURVIVAL ANALYSIS

Scenario: 3 players, 2 skips, 5 rounds
  Lives: 4
  Total Resources (lives + skips): 6
  Success Rate: 100.0%
  Verdict: ✓ CAN SURVIVE
  Confidence: HIGH

Scenario: 3 players, 2 skips, 10 rounds
  Lives: 4
  Total Resources (lives + skips): 6
  Success Rate: 0.8%
  Verdict: ✗ WILL LIKELY FAIL
  Confidence: HIGH

Scenario: 4 players, 1 skips, 8 rounds
  Lives: 5
  Total Resources (lives + skips): 6
  Success Rate: 0.0%
  Verdict: ✗ WILL LIKELY FAIL
  Confidence: HIGH

Scenario: 5 players, 3 skips, 15 rounds
  Lives: 6
  Total Resources (lives + skips): 9
  Success Rate: 0.0%
  Verdict: ✗ WILL LIKELY FAIL
  Confidence: HIGH

Scenario: 2 players, 5 skips, 10 rounds
  Lives: 3
  Total Resources (lives + skips): 8
  Success Rate: 95.3%
  Verdict: ✓ CAN SURVIVE
  Confidence: HIGH



# Question 5: Neuron Matrix State Transition

You have a 2D matrix of numbers representing neurons. Each neuron has a state (firing or not firing) and transitions to a new state based on its neighbors.

## Rules:

### Neuron States:
- **Firing neuron**: value > 0
- **Non-firing neuron**: value = 0

### State Transition Rules:
1. **Firing neuron** (value > 0):
   - If exactly 3 neighbors are firing → set to 6
   - Otherwise → keep current value

2. **Non-firing neuron** (value = 0):
   - If 0 or 1 neighbors are firing → decrement by 2 (cannot go below 0)
   - If more than 3 neighbors are firing → decrement by 1 (cannot go below 0)
   - Otherwise → keep current value

### Neighbors:
- A neuron's neighbors are the up to 8 surrounding cells (horizontal, vertical, and diagonal)
- Edge and corner cells have fewer neighbors

## Task:
Given an `input_state` matrix, compute and return the `next_state` matrix.

In [80]:
import numpy as np
from typing import List

def count_firing_neighbors(matrix: List[List[int]], row: int, col: int) -> int:
    """
    Count the number of firing neighbors (value > 0) for a given cell.
    
    Args:
        matrix: The 2D matrix of neuron states
        row: Row index of the cell
        col: Column index of the cell
    
    Returns:
        Number of firing neighbors
    """
    rows = len(matrix)
    cols = len(matrix[0])
    
    # All 8 possible neighbor directions (including diagonals)
    directions = [
        (-1, -1), (-1, 0), (-1, 1),  # top-left, top, top-right
        (0, -1),           (0, 1),   # left, right
        (1, -1),  (1, 0),  (1, 1)    # bottom-left, bottom, bottom-right
    ]
    
    firing_count = 0
    
    for dr, dc in directions:
        new_row = row + dr
        new_col = col + dc
        
        # Check if neighbor is within bounds
        if 0 <= new_row < rows and 0 <= new_col < cols:
            if matrix[new_row][new_col] > 0:
                firing_count += 1
    
    return firing_count

In [81]:
def compute_next_state(input_state: List[List[int]]) -> List[List[int]]:
    """
    Compute the next state of the neuron matrix based on transition rules.
    
    Args:
        input_state: 2D matrix of current neuron states
    
    Returns:
        2D matrix of next neuron states
    """
    rows = len(input_state)
    cols = len(input_state[0])
    
    # Create a new matrix for the next state
    next_state = [[0 for _ in range(cols)] for _ in range(rows)]
    
    for row in range(rows):
        for col in range(cols):
            current_value = input_state[row][col]
            firing_neighbors = count_firing_neighbors(input_state, row, col)
            
            if current_value > 0:
                # Firing neuron
                if firing_neighbors == 3:
                    next_state[row][col] = 6
                else:
                    next_state[row][col] = current_value
            else:
                # Non-firing neuron (value = 0)
                if firing_neighbors <= 1:
                    # Decrement by 2 (but cannot go below 0)
                    next_state[row][col] = max(0, current_value - 2)
                elif firing_neighbors > 3:
                    # Decrement by 1 (but cannot go below 0)
                    next_state[row][col] = max(0, current_value - 1)
                else:
                    # 2 or 3 neighbors firing - keep current value
                    next_state[row][col] = current_value
    
    return next_state

In [82]:
def print_matrix(matrix: List[List[int]], title: str = "Matrix"):
    """
    Pretty print a matrix.
    
    Args:
        matrix: 2D matrix to print
        title: Title for the matrix
    """
    print(f"\n{title}:")
    print("-" * 40)
    for row in matrix:
        print("  ", end="")
        for val in row:
            print(f"{val:3}", end=" ")
        print()
    print()

# Test Example 1: Simple 3x3 matrix
print("="*60)
print("TEST EXAMPLE 1: 3x3 Matrix")
print("="*60)

input_state_1 = [
    [0, 1, 0],
    [1, 2, 1],
    [0, 1, 0]
]

print_matrix(input_state_1, "Input State")

# Count neighbors for each cell
print("Firing neighbor counts:")
for i in range(3):
    print(f"  Row {i}: ", end="")
    for j in range(3):
        count = count_firing_neighbors(input_state_1, i, j)
        print(f"{count} ", end="")
    print()

next_state_1 = compute_next_state(input_state_1)
print_matrix(next_state_1, "Next State")

print("\nExplanation:")
print("  - Center cell (2, firing): has 4 firing neighbors → stays 2")
print("  - Top center (1, firing): has 2 firing neighbors → stays 1")
print("  - Corners (0, non-firing): have 2 firing neighbors → stay 0")
print("  - Sides (1, firing): have 2-3 firing neighbors → stay 1")

TEST EXAMPLE 1: 3x3 Matrix

Input State:
----------------------------------------
    0   1   0 
    1   2   1 
    0   1   0 

Firing neighbor counts:
  Row 0: 3 3 3 
  Row 1: 3 4 3 
  Row 2: 3 3 3 

Next State:
----------------------------------------
    0   6   0 
    6   2   6 
    0   6   0 


Explanation:
  - Center cell (2, firing): has 4 firing neighbors → stays 2
  - Top center (1, firing): has 2 firing neighbors → stays 1
  - Corners (0, non-firing): have 2 firing neighbors → stay 0
  - Sides (1, firing): have 2-3 firing neighbors → stay 1


In [83]:
# Test Example 2: Matrix with firing neurons that have exactly 3 neighbors
print("="*60)
print("TEST EXAMPLE 2: Firing neuron with exactly 3 firing neighbors")
print("="*60)

input_state_2 = [
    [5, 3, 0, 0],
    [2, 4, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 0]
]

print_matrix(input_state_2, "Input State")

# Analyze specific cell
target_row, target_col = 1, 1  # Cell with value 4
neighbors = count_firing_neighbors(input_state_2, target_row, target_col)
print(f"Cell [{target_row}][{target_col}] (value={input_state_2[target_row][target_col]}) has {neighbors} firing neighbors")

next_state_2 = compute_next_state(input_state_2)
print_matrix(next_state_2, "Next State")

print("\nExplanation:")
print(f"  - Cell [1][1] (value 4): has 5 firing neighbors → stays 4")
print(f"  - Cell [0][0] (value 5): has 3 firing neighbors → becomes 6 ✓")
print(f"  - Cell [2][1] (value 1): has 4 firing neighbors → stays 1")
print(f"  - Non-firing cells with 0-1 neighbors would decrement by 2 (but already 0)")

TEST EXAMPLE 2: Firing neuron with exactly 3 firing neighbors

Input State:
----------------------------------------
    5   3   0   0 
    2   4   1   0 
    0   1   0   0 
    0   0   0   0 

Cell [1][1] (value=4) has 5 firing neighbors

Next State:
----------------------------------------
    6   3   0   0 
    2   4   6   0 
    0   6   0   0 
    0   0   0   0 


Explanation:
  - Cell [1][1] (value 4): has 5 firing neighbors → stays 4
  - Cell [0][0] (value 5): has 3 firing neighbors → becomes 6 ✓
  - Cell [2][1] (value 1): has 4 firing neighbors → stays 1
  - Non-firing cells with 0-1 neighbors would decrement by 2 (but already 0)


In [84]:
# Test Example 3: Edge cases with non-firing neurons
print("="*60)
print("TEST EXAMPLE 3: Non-firing neuron behavior")
print("="*60)

# Create a scenario where non-firing neurons have different neighbor counts
input_state_3 = [
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

print_matrix(input_state_3, "Input State")

# Analyze the center non-firing cell
center_neighbors = count_firing_neighbors(input_state_3, 2, 2)
print(f"Center cell [2][2] (non-firing, value=0) has {center_neighbors} firing neighbors")

# Analyze corner non-firing cell
corner_neighbors = count_firing_neighbors(input_state_3, 0, 0)
print(f"Corner cell [0][0] (non-firing, value=0) has {corner_neighbors} firing neighbors")

next_state_3 = compute_next_state(input_state_3)
print_matrix(next_state_3, "Next State")

print("\nExplanation:")
print(f"  - Center [2][2] (non-firing): has 8 firing neighbors (>3) → max(0, 0-1) = 0")
print(f"  - Corner [0][0] (non-firing): has 1 firing neighbor (≤1) → max(0, 0-2) = 0")
print(f"  - Edge cells [0][2] (non-firing): has 3 firing neighbors (2-3) → stays 0")
print(f"  - Firing neurons: None have exactly 3 neighbors, so all stay same")

TEST EXAMPLE 3: Non-firing neuron behavior

Input State:
----------------------------------------
    0   0   0   0   0 
    0   1   1   1   0 
    0   1   0   1   0 
    0   1   1   1   0 
    0   0   0   0   0 

Center cell [2][2] (non-firing, value=0) has 8 firing neighbors
Corner cell [0][0] (non-firing, value=0) has 1 firing neighbors

Next State:
----------------------------------------
    0   0   0   0   0 
    0   1   1   1   0 
    0   1   0   1   0 
    0   1   1   1   0 
    0   0   0   0   0 


Explanation:
  - Center [2][2] (non-firing): has 8 firing neighbors (>3) → max(0, 0-1) = 0
  - Corner [0][0] (non-firing): has 1 firing neighbor (≤1) → max(0, 0-2) = 0
  - Edge cells [0][2] (non-firing): has 3 firing neighbors (2-3) → stays 0
  - Firing neurons: None have exactly 3 neighbors, so all stay same


In [85]:
# Test Example 4: Complex scenario with multiple state changes
print("="*60)
print("TEST EXAMPLE 4: Complex state transition")
print("="*60)

input_state_4 = [
    [3, 4, 1, 0],
    [2, 0, 5, 1],
    [1, 2, 0, 0],
    [0, 0, 1, 2]
]

print_matrix(input_state_4, "Input State")

print("Detailed neighbor analysis:")
for i in range(4):
    for j in range(4):
        val = input_state_4[i][j]
        neighbors = count_firing_neighbors(input_state_4, i, j)
        status = "firing" if val > 0 else "non-firing"
        print(f"  [{i}][{j}] val={val} ({status}): {neighbors} firing neighbors", end="")
        
        # Determine what happens
        if val > 0:
            if neighbors == 3:
                print(f" → becomes 6")
            else:
                print(f" → stays {val}")
        else:
            if neighbors <= 1:
                print(f" → max(0, {val}-2) = 0")
            elif neighbors > 3:
                print(f" → max(0, {val}-1) = 0")
            else:
                print(f" → stays {val}")

next_state_4 = compute_next_state(input_state_4)
print_matrix(next_state_4, "Next State")

TEST EXAMPLE 4: Complex state transition

Input State:
----------------------------------------
    3   4   1   0 
    2   0   5   1 
    1   2   0   0 
    0   0   1   2 

Detailed neighbor analysis:
  [0][0] val=3 (firing): 2 firing neighbors → stays 3
  [0][1] val=4 (firing): 4 firing neighbors → stays 4
  [0][2] val=1 (firing): 3 firing neighbors → becomes 6
  [0][3] val=0 (non-firing): 3 firing neighbors → stays 0
  [1][0] val=2 (firing): 4 firing neighbors → stays 2
  [1][1] val=0 (non-firing): 7 firing neighbors → max(0, 0-1) = 0
  [1][2] val=5 (firing): 4 firing neighbors → stays 5
  [1][3] val=1 (firing): 2 firing neighbors → stays 1
  [2][0] val=1 (firing): 2 firing neighbors → stays 1
  [2][1] val=2 (firing): 4 firing neighbors → stays 2
  [2][2] val=0 (non-firing): 5 firing neighbors → max(0, 0-1) = 0
  [2][3] val=0 (non-firing): 4 firing neighbors → max(0, 0-1) = 0
  [3][0] val=0 (non-firing): 2 firing neighbors → stays 0
  [3][1] val=0 (non-firing): 3 firing neighbors → s

In [86]:
# Test Example 5: Multiple iterations to see evolution
print("="*60)
print("TEST EXAMPLE 5: Multi-step evolution")
print("="*60)

# Start with a simple pattern
current_state = [
    [0, 0, 0, 0, 0],
    [0, 1, 2, 1, 0],
    [0, 2, 3, 2, 0],
    [0, 1, 2, 1, 0],
    [0, 0, 0, 0, 0]
]

print_matrix(current_state, "Initial State (Step 0)")

# Run 3 iterations
for step in range(1, 4):
    current_state = compute_next_state(current_state)
    print_matrix(current_state, f"State after Step {step}")
    
print("Observation: The pattern evolves as neurons fire and react to their neighbors.")

TEST EXAMPLE 5: Multi-step evolution

Initial State (Step 0):
----------------------------------------
    0   0   0   0   0 
    0   1   2   1   0 
    0   2   3   2   0 
    0   1   2   1   0 
    0   0   0   0   0 


State after Step 1:
----------------------------------------
    0   0   0   0   0 
    0   6   2   6   0 
    0   2   3   2   0 
    0   6   2   6   0 
    0   0   0   0   0 


State after Step 2:
----------------------------------------
    0   0   0   0   0 
    0   6   2   6   0 
    0   2   3   2   0 
    0   6   2   6   0 
    0   0   0   0   0 


State after Step 3:
----------------------------------------
    0   0   0   0   0 
    0   6   2   6   0 
    0   2   3   2   0 
    0   6   2   6   0 
    0   0   0   0   0 

Observation: The pattern evolves as neurons fire and react to their neighbors.


## Summary of Rules

Let's verify our implementation matches all the rules:

### Firing Neuron (value > 0):
- ✓ If exactly 3 neighbors are firing → set to 6
- ✓ Otherwise → keep current value

### Non-Firing Neuron (value = 0):
- ✓ If 0 or 1 neighbors are firing → decrement by 2 (cannot go below 0)
- ✓ If more than 3 neighbors are firing → decrement by 1 (cannot go below 0)
- ✓ If 2 or 3 neighbors are firing → keep current value (stays 0)

The implementation correctly handles all edge cases including:
- Cells at corners (3 neighbors)
- Cells at edges (5 neighbors)
- Interior cells (8 neighbors)
- All neurons update simultaneously based on the current state

# Question 6: Shortest Distance in a Tree

Given a tree represented as a dictionary and two nodes in the tree, find the shortest distance between the two nodes.

## Tree Representation:
The tree is represented as a dictionary where:
- Keys are node names
- Values are lists of children nodes

## Task:
Implement a function to find the shortest distance (number of edges) between two nodes in the tree.

## Follow-up Questions:
1. **Two sets of length 2**: Given `a = [a1, a2]` and `b = [b1, b2]`, find the shortest distance between any pair `(x, y)` where `x ∈ a` and `y ∈ b`
2. **Arbitrary-length sets**: Extend to handle arbitrary-length sets `a` and `b`

In [87]:
from typing import Dict, List, Optional
from collections import deque

# Example tree represented as a dictionary (parent -> children mapping)
tree = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['G', 'H'],
    'F': [],
    'G': [],
    'H': []
}

print("Tree Structure:")
print("="*50)
print("         A")
print("        / \\")
print("       B   C")
print("      / \\   \\")
print("     D   E   F")
print("        / \\")
print("       G   H")
print("\nTree Dictionary:")
for parent, children in tree.items():
    print(f"  {parent}: {children}")

Tree Structure:
         A
        / \
       B   C
      / \   \
     D   E   F
        / \
       G   H

Tree Dictionary:
  A: ['B', 'C']
  B: ['D', 'E']
  C: ['F']
  D: []
  E: ['G', 'H']
  F: []
  G: []
  H: []


In [88]:
def build_graph(tree: Dict[str, List[str]]) -> Dict[str, List[str]]:
    """
    Build an undirected graph (adjacency list) from the tree dictionary.
    The tree dict has parent -> children, we need bidirectional edges.
    
    Args:
        tree: Dictionary mapping nodes to their children
    
    Returns:
        Adjacency list with bidirectional edges
    """
    graph = {}
    
    # Initialize all nodes
    for node in tree:
        if node not in graph:
            graph[node] = []
    
    # Add bidirectional edges
    for parent, children in tree.items():
        for child in children:
            if child not in graph:
                graph[child] = []
            graph[parent].append(child)
            graph[child].append(parent)
    
    return graph

def find_shortest_distance(tree: Dict[str, List[str]], node1: str, node2: str) -> Optional[int]:
    """
    Find the shortest distance between two nodes in a tree using BFS.
    
    Args:
        tree: Dictionary representing the tree (parent -> children)
        node1: First node
        node2: Second node
    
    Returns:
        Shortest distance (number of edges) between the nodes, or None if no path exists
    """
    # Build bidirectional graph
    graph = build_graph(tree)
    
    # Check if nodes exist
    if node1 not in graph or node2 not in graph:
        return None
    
    # If same node, distance is 0
    if node1 == node2:
        return 0
    
    # BFS to find shortest path
    queue = deque([(node1, 0)])  # (current_node, distance)
    visited = {node1}
    
    while queue:
        current, dist = queue.popleft()
        
        # Check neighbors
        for neighbor in graph[current]:
            if neighbor == node2:
                return dist + 1
            
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, dist + 1))
    
    # No path found
    return None

In [89]:
# Test the shortest distance function
print("="*60)
print("TEST: Finding Shortest Distances")
print("="*60)

test_cases = [
    ('D', 'H'),  # D to H
    ('A', 'G'),  # A to G
    ('F', 'D'),  # F to D
    ('B', 'C'),  # B to C
    ('A', 'A'),  # Same node
    ('G', 'H'),  # Siblings
]

for node1, node2 in test_cases:
    distance = find_shortest_distance(tree, node1, node2)
    print(f"\nDistance from {node1} to {node2}: {distance}")
    
    # Show path explanation for some cases
    if (node1, node2) == ('D', 'H'):
        print("  Path: D -> B -> E -> H")
    elif (node1, node2) == ('A', 'G'):
        print("  Path: A -> B -> E -> G")
    elif (node1, node2) == ('F', 'D'):
        print("  Path: F -> C -> A -> B -> D")
    elif (node1, node2) == ('B', 'C'):
        print("  Path: B -> A -> C")
    elif (node1, node2) == ('G', 'H'):
        print("  Path: G -> E -> H")

TEST: Finding Shortest Distances

Distance from D to H: 3
  Path: D -> B -> E -> H

Distance from A to G: 3
  Path: A -> B -> E -> G

Distance from F to D: 4
  Path: F -> C -> A -> B -> D

Distance from B to C: 2
  Path: B -> A -> C

Distance from A to A: 0

Distance from G to H: 2
  Path: G -> E -> H


## Follow-up 1: Shortest Distance Between Two Sets (Length 2)

Given two sets `a = [a1, a2]` and `b = [b1, b2]`, find the shortest distance between any pair `(x, y)` where `x ∈ a` and `y ∈ b`.

In [90]:
def shortest_distance_two_sets_length2(tree: Dict[str, List[str]], 
                                       a: List[str], 
                                       b: List[str]) -> Optional[int]:
    """
    Find shortest distance between two sets of nodes (both of length 2).
    
    Args:
        tree: Dictionary representing the tree
        a: List of 2 nodes [a1, a2]
        b: List of 2 nodes [b1, b2]
    
    Returns:
        Minimum distance among all pairs (x, y) where x in a and y in b
    """
    if len(a) != 2 or len(b) != 2:
        raise ValueError("Both sets must have exactly 2 elements")
    
    min_distance = float('inf')
    best_pair = None
    
    # Check all 4 combinations: (a1,b1), (a1,b2), (a2,b1), (a2,b2)
    for node_a in a:
        for node_b in b:
            dist = find_shortest_distance(tree, node_a, node_b)
            if dist is not None and dist < min_distance:
                min_distance = dist
                best_pair = (node_a, node_b)
    
    if min_distance == float('inf'):
        return None
    
    print(f"  Best pair: {best_pair[0]} -> {best_pair[1]}")
    return min_distance

# Test with sets of length 2
print("="*60)
print("FOLLOW-UP 1: Two Sets of Length 2")
print("="*60)

test_cases_sets = [
    (['D', 'F'], ['G', 'H']),  # Left leaves vs right leaves under E
    (['A', 'B'], ['F', 'G']),  # Top nodes vs bottom nodes
    (['D', 'E'], ['C', 'F']),  # Left side vs right side
]

for a, b in test_cases_sets:
    print(f"\na = {a}, b = {b}")
    distance = shortest_distance_two_sets_length2(tree, a, b)
    print(f"Shortest distance: {distance}")

FOLLOW-UP 1: Two Sets of Length 2

a = ['D', 'F'], b = ['G', 'H']
  Best pair: D -> G
Shortest distance: 3

a = ['A', 'B'], b = ['F', 'G']
  Best pair: A -> F
Shortest distance: 2

a = ['D', 'E'], b = ['C', 'F']
  Best pair: D -> C
Shortest distance: 3


## Follow-up 2: Shortest Distance Between Arbitrary-Length Sets

Extend to handle arbitrary-length sets `a` and `b`.

In [91]:
def shortest_distance_arbitrary_sets(tree: Dict[str, List[str]], 
                                     a: List[str], 
                                     b: List[str]) -> Optional[int]:
    """
    Find shortest distance between two arbitrary-length sets of nodes.
    
    Args:
        tree: Dictionary representing the tree
        a: List of nodes (arbitrary length)
        b: List of nodes (arbitrary length)
    
    Returns:
        Minimum distance among all pairs (x, y) where x in a and y in b,
        along with the best pair
    """
    if not a or not b:
        return None
    
    min_distance = float('inf')
    best_pair = None
    
    # Check all combinations
    for node_a in a:
        for node_b in b:
            dist = find_shortest_distance(tree, node_a, node_b)
            if dist is not None and dist < min_distance:
                min_distance = dist
                best_pair = (node_a, node_b)
    
    if min_distance == float('inf'):
        return None
    
    print(f"  Best pair: {best_pair[0]} -> {best_pair[1]}")
    return min_distance

# Test with arbitrary-length sets
print("="*60)
print("FOLLOW-UP 2: Arbitrary-Length Sets")
print("="*60)

test_cases_arbitrary = [
    (['D', 'F', 'G'], ['H']),           # 3 nodes vs 1 node
    (['A'], ['D', 'E', 'F', 'G', 'H']), # 1 node vs 5 nodes
    (['D', 'F'], ['G', 'H', 'C']),      # 2 nodes vs 3 nodes
    (['A', 'B', 'C'], ['D', 'E', 'F']), # 3 nodes vs 3 nodes
]

for a, b in test_cases_arbitrary:
    print(f"\na = {a} (size {len(a)})")
    print(f"b = {b} (size {len(b)})")
    distance = shortest_distance_arbitrary_sets(tree, a, b)
    print(f"Shortest distance: {distance}")

FOLLOW-UP 2: Arbitrary-Length Sets

a = ['D', 'F', 'G'] (size 3)
b = ['H'] (size 1)
  Best pair: G -> H
Shortest distance: 2

a = ['A'] (size 1)
b = ['D', 'E', 'F', 'G', 'H'] (size 5)
  Best pair: A -> D
Shortest distance: 2

a = ['D', 'F'] (size 2)
b = ['G', 'H', 'C'] (size 3)
  Best pair: F -> C
Shortest distance: 1

a = ['A', 'B', 'C'] (size 3)
b = ['D', 'E', 'F'] (size 3)
  Best pair: B -> D
Shortest distance: 1


## Optimization: Multi-Source BFS

For large sets, we can optimize by using multi-source BFS instead of checking all pairs individually.

**Time Complexity Comparison:**
- Naive approach: O(|a| × |b| × (V + E)) where V is vertices, E is edges
- Multi-source BFS: O((|a| + |b|) × (V + E))

The multi-source BFS approach is much more efficient when sets are large.

In [92]:
def shortest_distance_optimized(tree: Dict[str, List[str]], 
                                a: List[str], 
                                b: List[str]) -> Optional[int]:
    """
    Optimized version using multi-source BFS.
    Start BFS from all nodes in set 'a' simultaneously and find first node in set 'b'.
    
    Args:
        tree: Dictionary representing the tree
        a: List of source nodes
        b: List of target nodes
    
    Returns:
        Minimum distance from any node in a to any node in b
    """
    if not a or not b:
        return None
    
    # Build graph
    graph = build_graph(tree)
    
    # Check for nodes that exist in both sets
    overlap = set(a) & set(b)
    if overlap:
        print(f"  Found overlap: {overlap}")
        return 0
    
    # Multi-source BFS: start from all nodes in 'a'
    queue = deque()
    visited = set()
    b_set = set(b)
    
    # Initialize queue with all nodes from set 'a' at distance 0
    for node in a:
        if node in graph:
            queue.append((node, 0, node))  # (current_node, distance, source_node)
            visited.add(node)
    
    # BFS
    while queue:
        current, dist, source = queue.popleft()
        
        # Check if we reached any node in set b
        if current in b_set:
            print(f"  Best pair: {source} -> {current}")
            return dist
        
        # Explore neighbors
        for neighbor in graph[current]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, dist + 1, source))
    
    return None

# Test optimized version
print("="*60)
print("OPTIMIZED VERSION: Multi-Source BFS")
print("="*60)

for a, b in test_cases_arbitrary:
    print(f"\na = {a} (size {len(a)})")
    print(f"b = {b} (size {len(b)})")
    distance = shortest_distance_optimized(tree, a, b)
    print(f"Shortest distance: {distance}")

OPTIMIZED VERSION: Multi-Source BFS

a = ['D', 'F', 'G'] (size 3)
b = ['H'] (size 1)
  Best pair: G -> H
Shortest distance: 2

a = ['A'] (size 1)
b = ['D', 'E', 'F', 'G', 'H'] (size 5)
  Best pair: A -> D
Shortest distance: 2

a = ['D', 'F'] (size 2)
b = ['G', 'H', 'C'] (size 3)
  Best pair: F -> C
Shortest distance: 1

a = ['A', 'B', 'C'] (size 3)
b = ['D', 'E', 'F'] (size 3)
  Best pair: B -> D
Shortest distance: 1


## Summary

We've implemented three approaches to find shortest distances in a tree:

1. **Basic Approach**: Find shortest distance between two individual nodes
   - Uses BFS from one node to another
   - Time: O(V + E)

2. **Naive Set Approach**: Find shortest distance between two sets
   - Checks all pairs (x, y) where x ∈ a, y ∈ b
   - Time: O(|a| × |b| × (V + E))

3. **Optimized Multi-Source BFS**: Efficient version for large sets
   - Starts BFS from all nodes in set 'a' simultaneously
   - Stops when any node in set 'b' is reached
   - Time: O((|a| + |b|) × (V + E))
   
**When to use each:**
- Use the basic approach for single node-to-node queries
- Use naive approach when sets are small (|a| × |b| < 10)
- Use optimized approach for larger sets or repeated queries

In [93]:
print(len('fff'))

3
